<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [17]:
# _*_ conding: utf-8 _*_
import requests
import json
import re
from requests.exceptions import RequestException
from bs4 import BeautifulSoup
import pandas as pd

def get_url(start_page,jobname):
    try:
        headers = {
            'User-Agent': 'User-Agent  Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'zh-CN'
        }
        params = {
            'start': start_page,
            'pageSize': 90,
            'cityId': 765,
           'workExperience': -1,
            'education': -1,
            'companyType': -1,
            'employmentType': -1,
            'jobWelfareTag': -1,
            'kw': jobname,
            'kt': 3,
            '_v': 0.49167600,
            'x-zp-page-request-id': 'd7e410fe10de4284a911bd59aa7c39d2-1547036334715-373301'
        }
        url = 'https://fe-api.zhaopin.com/c/i/sou?'
        html = requests.get(url, headers=headers,params=params)
        if html.status_code == 200:
            results = json.loads(html.text).get('data').get('results')
            URL_list = []
            for job in results:
                positionURL = job.get('positionURL')
                URL_list.append(positionURL)
            return URL_list
        return None
    except RequestException:
        return None

In [18]:
def get_page_parse(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return  BeautifulSoup(response.text, 'lxml')            
        return None
    except RequestException:
        return None

In [66]:
def parse_page(response):
    pos_name = response.h1.string
    salary = response.strong.string
    company = response.select('div.company a')[0].string
    city = response.select('div.info-three span')[0].string
    experence = response.select('div.info-three span')[1].string
    edu = response.select('div.info-three span')[2].string
    num = response.select('div.info-three span')[3].string
    job_list = response.select('div.pos-ul div div')
    job_text = []
    for job in job_list:
        job_text.append(job.get_text())
    #     print(pos_name)
    address = response.select('p.add-txt')[0].get_text()
    company_indu = response.select('ul.promulgator-ul li strong')[0].get_text()
    company_type = response.select('ul.promulgator-ul li strong')[1].get_text()
    company_num = response.select('ul.promulgator-ul li strong')[2].get_text()
    company_add = response.select(
        'ul.promulgator-ul li strong')[4].get_text().strip()
    return {
        'pos_name': pos_name,
        'salary': salary,
        'company': company,
        'city': city,
        'experence': experence,
        'edu': edu,
        'num': num,
#         'job_text': job_text,
        'address': address,
        'company_indu': company_indu,
        'company_type': company_type,
        'company_num': company_num,
#         'company_add': company_add
    }

In [121]:
import pymongo
mongo_url = 'localhost'
mongo_db = 'zhilian'
client = pymongo.MongoClient(mongo_url)
db = client[mongo_db]
def save_to_mongo(MONGO_TABLE,text):
    if db[MONGO_TABLE].insert_one(text):
        print('存储到MONGODB成功')
        return True
    return False

In [125]:
def save_to_json(result):
    with open('zhilian.txt','a') as f:
        f.write(json.dumps(result,ensure_ascii = False)+ '\n')

In [130]:
def main(start_page,kw):
    positionURL = get_url(start_page=start_page,jobname=kw)
    for url in positionURL:
        print(url)
        response = get_page(url)
        result = parse_page(response)
        save_to_json(result)
        save_to_mongo('zhilian',result)

In [132]:
resut_df = pd.DataFrame()
if __name__=='__main__':
    for i in range(10):
        data = main(start_page=i*90,kw='Python')

https://jobs.zhaopin.com/CC574811320J00144157805.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC574811320J00238507305.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00247230905.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC296721081J00177997105.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC296721081J00174952505.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC296721081J00174940805.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00246419505.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC296721081J00175002805.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC296721081J00177153105.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00248220205.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00252754905.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00242518505.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00252752305.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00248651005.htm
存储到MONGODB成功
https://jobs.zhaopin.com/120143601284535.htm
存储到MONGODB成功
https://jobs.zhaop

存储到MONGODB成功
https://jobs.zhaopin.com/644924620250125.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC842910190J00176413904.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC601964480J00287997901.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC690553137J00183268102.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC842910190J00173666004.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC275310134J00133707806.htm
存储到MONGODB成功
https://jobs.zhaopin.com/153170415251258.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC275310134J00112700406.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC638504025J00162686403.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC208751619J00182047502.htm
存储到MONGODB成功
https://jobs.zhaopin.com/507087822250388.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC275310134J00091399306.htm
存储到MONGODB成功
https://jobs.zhaopin.com/613306221250186.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC498574036J00059913904.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC613306221J00226106907.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC64

存储到MONGODB成功
https://jobs.zhaopin.com/CC196250025J00098257803.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC268769911J00180757108.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC358101529J00300191701.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC131486150J00224476807.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC252519037J00101264414.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CZ719734740J00069912703.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CZ742128140J00299137801.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC469912010J00187552202.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC278441814J00184969806.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC268769911J00186227108.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CZ632185180J00235822007.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC268769911J00182377208.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC268769911J00180832008.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC252519037J00097832314.htm
存储到MONGODB成功
https://jobs.zhaopin.com/CC508620126J00240745205.htm
存储到MONGODB

UnicodeEncodeError: 'gbk' codec can't encode character '\xa0' in position 30: illegal multibyte sequence

In [119]:
resut_df

,0
0,五险一金+数据分析师
1,4000-8000元/月
2,深圳市臻信资产管理有限公司
3,深圳
4,经验不限
5,本科
6,招1人
7,深圳市罗湖区桂园街道宝安南路
8,信托/担保/拍卖/典当
9,民营


In [253]:
# get_page('https://jobs.zhaopin.com/CC675886524J00248602905.htm')